In [1]:
!pip install sodapy

    100% |████████████████████████████████| 829kB 744kB/s ta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/davidplewis/Library/Caches/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built future
distributed 1.21.8 requires msgpack, which is not installed.


In [3]:
# from https://github.com/socrata/dev.socrata.com/blob/39c6581986466edb5e7f72f5beea5ce69238f8de/snippets/pandas.py

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# First 50000 results, returned as JSON from API 
# Connverted to Python list of dictionaries by sodapy.
# Column names converted to snake case, special chars removed
# Dates and location formatted
results = client.get("r5kz-chrr", limit=50000)

# Convert to pandas DataFrame
licenses = pd.DataFrame.from_records(results)

In [4]:
# download remaining (limit 50000 / call)
start = 50000
while results:
    print(start)
    results = client.get("r5kz-chrr", limit=50000, offset=start)
    licenses = licenses.append(pd.DataFrame.from_records(results))
    start += 50000


50000
100000
150000
200000
250000
300000
350000
400000
450000
500000


/Users/davidplewis/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


550000
600000
650000
700000
750000
800000
850000
900000
950000


In [11]:
# Query data to get an idea of what information is present
licenses.head()

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
5                 NaN
6                 NaN
7                 NaN
8                 NaN
9                 NaN
10                NaN
11                NaN
12                NaN
13                NaN
14                NaN
15                NaN
16                NaN
17                NaN
18                NaN
19                NaN
20                NaN
21                NaN
22                NaN
23                NaN
24                NaN
25                NaN
26                NaN
27                NaN
28                NaN
29                NaN
             ...     
38015    41.997621491
38016    41.791095234
38017             NaN
38018    41.808646761
38019    41.954124675
38020    41.690543445
38021    41.783767852
38022    41.956998501
38023             NaN
38024     41.88743405
38025    41.953685818
38026    41.849379215
38027        41.95577
38028    41.889832917
38029    4

In [7]:
type(licenses.iloc[40].application_requirements_complete)


str

In [8]:
# Drop duplicates
licenses.drop_duplicates("id", inplace=True)

In [7]:
# Drop rows with missing license term start date?
# Drop rows with missing license term expiration date?
# drop application types C_CAPA and C_SBA?business_licenses.to_csv("../../DATA/business_licenses")

In [9]:
import os.path
root_path = os.path.dirname(os.getcwd())

# Save result
licenses.to_csv(os.path.join(root_path, "DATA/business_licenses.csv"), index=False)